## 🪄 Install `raga-testing-platform` library

In [ ]:
pip install -U raga-testing-platform

### Put your `ACCESS KEY & SECRET KEY`

In [ ]:
# Define the access key, secret key, and host
ACCESS_KEY = "Paste_your_access_key_here"
SECRET_KEY = "Paste_your_secret_key_here"
HOST = "https://backend.raga-xai.satsure.co"

### Import All raga lib from raga module

In [ ]:
from raga import *
import datetime

### 🪄 Set up a Run

In [ ]:
project_name = "Satellite Imagery"
run_name = f"Demo-Run-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
test_session = TestSession(project_name=project_name,run_name=run_name, access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST)

### 👟 Run Drift Semantic Segmentation 

In [ ]:
rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold=21.0)

edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name="Drift-detection-test",
                                           train_dataset_name="grasslands-final",
                                           field_dataset_name="barrenlands-final",
                                           train_embed_col_name="ImageEmbedding",
                                           field_embed_col_name = "ImageEmbedding",
                                           output_type = "semantic_segmentation",
                                           level = "image",
                                           rules = rules)


test_session.add(edge_case_detection)

test_session.run()

### 👟 Run Failure Mode Analysis Semantic Segmentation - Cluster Level

In [ ]:
rules = FMARules()
rules.add(metric = "F1Score",  metric_threshold = 0.86, label = "ALL", type="label", background_label="Unlabelled", include_background=False)
rules.add(metric = "Precision",  metric_threshold = 0.86, label = "ALL", type="label", background_label="Unlabelled", include_background=True)
rules.add(metric = "wIoU", metric_threshold = 0.5, weights={"Road": 100, "Unlabelled": 1}, type="label", label = "ALL")

cls_default = clustering(method="k-means", embedding_col="ImageVectorsM1", level="image", args= {"numOfClusters": 7})

edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = "satellite_image_dataset",
                                            test_name = "Test",
                                            model = "ModelB",
                                            gt = "ModelA",
                                            rules = rules,
                                            output_type="semantic_segmentation",
                                            type="embedding",
                                            clustering=cls_default
                                            )

test_session.add(edge_case_detection)

test_session.run()

### 👟 Run Labeling Quality Semantic Segmentation Test

In [ ]:
rules = LQRules()
rules.add(metric="mistake_score", label=["ALL"], metric_threshold=0.065)

edge_case_detection = labelling_quality_test(test_session=test_session,
                                            dataset_name = "satsure-area-dataset-full-v1",
                                            test_name = "Labeling Quality Test",
                                            type = "labelling_consistency",
                                            output_type="semantic_segmentation",
                                            mistake_score_col_name = "MistakeScores",
                                            rules = rules)
test_session.add(edge_case_detection)

test_session.run()

Please switch to http://13.126.220.245:3000/projects